# Chuyển oco2 nc4 thành csv


In [ ]:
# thư viện cần thiết
import xarray as xr
import pandas as pd
import glob
import os
import geopandas as gpd
from shapely.geometry import Point

## Xác định file .nc4 và đọc dữ liệu bằng xarray


In [ ]:
# Xác định file .nc4 và đọc dữ liệu bằng xarray
# Đường dẫn tới file .nc4
file_path = r"E:\DownloadData\co2_nasa\data\oco2_nc4_2020\oco2_LtCO2_200101_B11210Ar_240913160328s.nc4"  # <-- sửa lại tên file thật

# Mở file .nc4
ds = xr.open_dataset(file_path)

# In thông tin toàn bộ dataset
print(ds)

# In danh sách các biến
print("\n🔹 Danh sách biến:")
print(list(ds.data_vars))

# In các chiều dữ liệu
print("\n🔹 Các chiều:")
print(list(ds.dims))

# In thuộc tính toàn cục
print("\n🔹 Global attributes:")
print(ds.attrs)

# Đóng file sau khi đọc
ds.close()


## Gộp dữ liệu nc4 vào file csv

In [ ]:
# Đường dẫn đến thư mục chứa các file .nc4
folder_path = "E:\\DownloadData\\co2_nasa\\data\\oco2_nc4_2020\\*.nc4"

# Lấy danh sách tất cả các file .nc4
file_list = glob.glob(folder_path)

# Danh sách lưu DataFrame từ mỗi file
df_list = []

In [ ]:
for file in file_list:
    filename = os.path.basename(file)
    date_str = filename.split('_')[2]
    full_date = f"20{date_str[:2]}-{date_str[2:4]}-{date_str[4:6]}"
    try:
        ds = xr.open_dataset(file)
        df = pd.DataFrame({
            "file_date": full_date,
            "latitude": ds["latitude"].values,
            "longitude": ds["longitude"].values,
            "sza": ds["solar_zenith_angle"].values,
            "vza": ds["sensor_zenith_angle"].values,
            "xco2": ds["xco2"].values,
            "xco2_sigma": ds["xco2_uncertainty"].values,
            "xco2_qual": ds["xco2_quality_flag"].values,
            "time": ds["time"].values,
        })
        df_list.append(df)
    except Exception as e:
        print(f"❌ Lỗi với file {file}: {e}")

# Kết hợp tất cả DataFrame thành một DataFrame duy nhất
df_final = pd.concat(df_list, ignore_index=True)

# Xuất CSV
output_csv = "oco2_all_2020.csv"
df_final.to_csv(output_csv, index=False)

print(f"✅ Xuất dữ liệu thành công ra {output_csv}!")
print(f"📂 Tổng số file đã xử lý: {len(file_list)}")
print(f"📊 Tổng số dòng dữ liệu: {len(df_final)}")


## Xuất dữ liệu CO2 từ OCO2_all lọc theo ranh giới Việt Nam sang file OCO2_vietnam


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [ ]:
# Đường dẫn
input_csv_path = r"E:\DownloadData\co2_nasa\data\oco2_csv\oco2_all_2020.csv"
shapefile_path = r"E:\RanhGioi\VNM_adm\gadm41_VNM_0.shp"
output_csv_path = r"E:\DownloadData\co2_nasa\data\oco2_csv\oco2_vietnam_2020.csv"

# Đọc shapefile ranh giới Việt Nam
gdf_vn = gpd.read_file(shapefile_path)
if gdf_vn.crs != "EPSG:4326":
    gdf_vn = gdf_vn.to_crs(epsg=4326)

# Lấy bounding box để lọc sơ bộ
min_lon, min_lat, max_lon, max_lat = gdf_vn.total_bounds

# Nếu file đã tồn tại thì xóa để ghi mới
if os.path.exists(output_csv_path):
    os.remove(output_csv_path)

# Đọc và xử lý từng phần nhỏ
chunk_size = 500_000  # có thể thay đổi tùy theo RAM
chunk_idx = 0

for chunk in pd.read_csv(input_csv_path, chunksize=chunk_size):
    chunk_idx += 1
    print(f"🔄 Đang xử lý phần {chunk_idx}...")

    # Lọc sơ bộ theo bounding box
    chunk = chunk[
        (chunk["longitude"] >= min_lon) & (chunk["longitude"] <= max_lon) &
        (chunk["latitude"] >= min_lat) & (chunk["latitude"] <= max_lat)
    ]

    # Tạo GeoDataFrame từ chunk đã lọc
    geometry = [Point(xy) for xy in zip(chunk["longitude"], chunk["latitude"])]
    gdf_chunk = gpd.GeoDataFrame(chunk, geometry=geometry, crs="EPSG:4326")

    # Lọc chính xác bằng spatial join
    gdf_filtered = gpd.sjoin(gdf_chunk, gdf_vn, predicate="within", how="inner")

    # Xóa các cột không cần
    gdf_filtered.drop(columns=["geometry", "index_right"], errors="ignore", inplace=True)

    # Ghi kết quả vào file (append)
    gdf_filtered.to_csv(output_csv_path, mode='a', index=False, header=not os.path.exists(output_csv_path))

    print(f"✅ Đã xử lý xong phần {chunk_idx}, ghi {len(gdf_filtered)} dòng.")

print("🎉 Hoàn tất lọc toàn bộ dữ liệu!")

## Lọc dữ liệu theo xco2_qual 

In [ ]:
import pandas as pd

df = pd.read_csv(r"E:\DownloadData\co2_nasa\data\oco2_csv\oco2_vietnam_2020.csv")

# Kiểm tra cột có tên đúng không
print(df.columns)

# Kiểm tra giá trị duy nhất trong cột xco2_qual
print(df['xco2_qual'].unique())


In [ ]:
import pandas as pd

# Đọc file
df = pd.read_csv(r"E:\DownloadData\co2_nasa\data\oco2_csv\oco2_vietnam_2020.csv")

# Lọc các dòng có xco2_qual = 0
filtered_df = df[df['xco2_qual'] == 0.0]  # dùng 0.0 hoặc 0 đều được

# Kiểm tra số dòng lọc được
print(f"Số dòng sau khi lọc xco2_qual = 0: {len(filtered_df)}")

# Xuất ra CSV
filtered_df.to_csv(r"E:\DownloadData\co2_nasa\data\oco2_csv\vn_xco2_qual0.csv", index=False)
